# Create recount3 model


Marc Subirana-Granés (2024)

Description


# Load libraries/modules


In [12]:
# Load libraries/modules
library(recount3)
library(PLIER)
library(here)
library(dplyr)
library(rhdf5)
library(HDF5Array)
library(glmnet)
library(biomaRt)

# Load PLIER pathway and cell type data
data(bloodCellMarkersIRISDMAP)
data(svmMarkers)
data(canonicalPathways)
data(chemgenPathways)
data(oncogenicPathways)
data(xCell)
data(immunePathways)
chr21_pathway <- readRDS(here::here('output/pathways/chr21_pathway.rds'))

#delayedPLIER functions from repo
path_script_funcs = '/home/msubirana/Documents/pivlab/DelayedPLIER/funcs.R'
source(path_script_funcs)

# Load data


In [13]:
# Define output nb path
output_nb_path <- here("output/nbs/create_recount3_model")

# Create directory if it doesn't exist
if (!dir.exists(output_nb_path)) {
  dir.create(output_nb_path, recursive = TRUE)
}

## Download recount3


In [14]:
available_samples()

2024-06-21 18:36:08.219283 caching file sra.recount_project.MD.gz.

2024-06-21 18:36:08.73041 caching file gtex.recount_project.MD.gz.

2024-06-21 18:36:09.282253 caching file tcga.recount_project.MD.gz.



external_id,project,organism,file_source,date_processed,project_home,project_type
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
SRR5579327,SRP107565,human,sra,2019-10-01,data_sources/sra,data_sources
SRR5579328,SRP107565,human,sra,2019-10-01,data_sources/sra,data_sources
SRR5579329,SRP107565,human,sra,2019-10-01,data_sources/sra,data_sources
SRR5579330,SRP107565,human,sra,2019-10-01,data_sources/sra,data_sources
SRR5579331,SRP107565,human,sra,2019-10-01,data_sources/sra,data_sources
SRR5579332,SRP107565,human,sra,2019-10-01,data_sources/sra,data_sources
SRR5579333,SRP107565,human,sra,2019-10-01,data_sources/sra,data_sources
SRR5579334,SRP107565,human,sra,2019-10-01,data_sources/sra,data_sources
SRR5579335,SRP107565,human,sra,2019-10-01,data_sources/sra,data_sources


# Qiwen Hu - 2017

# Processing all recounts datasets

# genes expression are normalized by RPKM

# It can be run from the command line using

# Rscript recount2/1-get_all_recount_dataset.R

#

# Output:

# Normalized gene expression for each sample

`%>%` <- dplyr::`%>%`
library(recount)

# Get RPKM value for each gene - adapted from recount package

getRPKM <- function(rse, length_var = 'bp_length', mapped_var = NULL) {

# Computes the RPKM value for each gene in the sample.

#

# Args:

# rse: A RangedSummarizedExperiment-class object in recount package

# length_var: A length 1 character vector with the column name from rowData(rse) that has

# the coding length. For gene level objects from recount this is bp_length. If

# NULL, then it will use width(rowRanges(rse)) which should be used for exon RSEs.

# mapped_var: A length 1 character vector with the column name from colData(rse) that has

# the number of reads mapped. If NULL (default) then it will use the column

# sums of the counts matrix

# Returns:

# RPKM value for each sample

if(!is.null(mapped_var)){
mapped <- colData(rse)[, mapped_var]
} else {
mapped <- colSums(assays(rse)$counts) 
  } 
  bg <- matrix(mapped, ncol = ncol(rse), nrow = nrow(rse), byrow = TRUE) 
  if(!is.null(length_var)){
    len <- rowData(rse)[, length_var] 
  } else {
    len <- width(rowRanges(rse))
  }
  wid <- matrix(len, nrow = nrow(rse), ncol = ncol(rse), byrow = FALSE) 
  rpkm <- assays(rse)$counts / (wid/1000) / (bg/1e6)
return(rpkm)
}

data.dir <- file.path("recount2", "data")
dir.create(data.dir, recursive = TRUE, showWarnings = FALSE)

# Get all samples from recount database

metasample.sra <- all_metadata(subset = "sra", verbose = TRUE)
metasample.sra <- as.data.frame(metasample.sra)

# Remove samples without description

metadata.nonempty <- metasample.sra[!is.na(metasample.sra$characteristics), ]
included.sample.list <- unique(metadata.nonempty$project)

# Download all recount2 samples in included.sample.list

lapply(included.sample.list,
function(x) download_study(x, type = "rse-gene",
outdir = file.path(data.dir, x)))

# get RPKM for each experiment and add to list

rpkm.list <- list()
for(experiment in included.sample.list) {
load(file.path(data.dir, experiment, 'rse_gene.Rdata'))
rpkm <- as.data.frame(getRPKM(rse_gene))
rpkm$id <- rownames(rpkm)
rpkm.list[[experiment]] <- rpkm
}

# combine experiments -- this is the most memory efficient way to go about this

# that I've found -- will need to drop extraneous gene id columns

rpkm.df <- do.call(base::cbind, c(rpkm.list, by = "id"))
rpkm.df <- rpkm.df %>% dplyr::select(-dplyr::ends_with("id"))
rpkm.df <- tibble::rownames_to_column(rpkm.df, "ENSG")

# drop last column "by" -- information about what was used with base::cbind

rpkm.df <- rpkm.df %>% dplyr::select(-by)

# save to file

saveRDS(rpkm.df, file = file.path("recount2", "recount_rpkm.RDS"))


In [15]:
human_projects <- available_projects("human")
human_projects <- human_projects[1:25,]

2024-06-21 18:36:10.640203 caching file sra.recount_project.MD.gz.

2024-06-21 18:36:11.104776 caching file gtex.recount_project.MD.gz.

2024-06-21 18:36:11.54756 caching file tcga.recount_project.MD.gz.



In [16]:
human_projects
sum(human_projects$n_samples)

,project,organism,file_source,project_home,project_type,n_samples
,<chr>,<chr>,<chr>,<chr>,<chr>,<table[1d]>
1,SRP107565,human,sra,data_sources/sra,data_sources,216
2,SRP149665,human,sra,data_sources/sra,data_sources,4
3,SRP017465,human,sra,data_sources/sra,data_sources,23
4,SRP119165,human,sra,data_sources/sra,data_sources,6
5,SRP133965,human,sra,data_sources/sra,data_sources,12
6,SRP096765,human,sra,data_sources/sra,data_sources,7
7,SRP124965,human,sra,data_sources/sra,data_sources,12
8,SRP189165,human,sra,data_sources/sra,data_sources,15
9,SRP050365,human,sra,data_sources/sra,data_sources,10


[1] 661

In [17]:
recount3_h5 <- file.path(output_nb_path, "recount3_dataset_tpm.h5")
h5createFile(recount3_h5)

In [18]:
ensembl <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")

In [19]:
# Define the function to load data and calculate TPM
process_project <- function(proj_info) {
    # Load project data
    rse <- recount3::create_rse(proj_info)
    
    # Ensure 'counts' field exists and is set to 'raw_counts' if necessary
    if (!"counts" %in% names(assays(rse))) {
        assays(rse)$counts <- assays(rse)$raw_counts
        assays(rse)$raw_counts <- NULL
    }
    
    # Calculate TPM and return the TPM matrix
    assays(rse)$TPM <- recount::getTPM(rse)
    return(assays(rse)$TPM)
}

# Initialize the HDF5 file and dataset
h5createFile(recount3_h5)

# Define initial dimensions assuming the first project has the correct dimensions
initial_proj_info <- human_projects[1, , drop = FALSE]
initial_matrix <- process_project(initial_proj_info)

rownames(initial_matrix) <- gsub("\\..*$", "", rownames(initial_matrix))
gene_ids <- rownames(initial_matrix)

gene_info <- getBM(attributes = c("ensembl_gene_id", "hgnc_symbol"),
                   filters = "ensembl_gene_id",
                   values = gene_ids,
                   mart = ensembl)

rownames(initial_matrix) <- gsub("\\..*$", "", rownames(initial_matrix))
    df_initial_matrix = data.frame(initial_matrix)
    df_initial_matrix <- df_initial_matrix %>%
    tibble::rownames_to_column(var = "ensembl_gene_id")
    # filter to remove genes without a gene symbol
    gene_info <- gene_info %>% dplyr::filter(complete.cases(.))
    # add gene symbols to expression df
    df_initial_matrix <- dplyr::inner_join(gene_info, df_initial_matrix, 
                                by = "ensembl_gene_id")
    df_initial_matrix <- df_initial_matrix %>%
        dplyr::filter(hgnc_symbol != '') %>%
        distinct(hgnc_symbol, .keep_all = TRUE) %>%
        tibble::column_to_rownames(var = "hgnc_symbol") %>%
        subset(select = -c(ensembl_gene_id))

    initial_matrix <- as.matrix(df_initial_matrix)

total_rows <- nrow(initial_matrix)

total_columns <- sum(sapply(1:nrow(human_projects), function(i) {
    ncol(process_project(human_projects[i, , drop = FALSE]))
}))

h5createDataset(recount3_h5, "count", 
                dims = c(total_rows, total_columns), 
                maxdims = c(total_rows, total_columns), 
                storage.mode = "double", 
                chunk = c(total_rows, 1000)) 

# Process and write each matrix sequentially
start_col <- 1

all_colnames <- c()

for (i in 1:nrow(human_projects)) {

    proj_info <- human_projects[i, , drop = FALSE]
    matrix <- process_project(proj_info)

    # id to symbols

    rownames(matrix) <- gsub("\\..*$", "", rownames(matrix))
    df_matrix = data.frame(matrix)
    df_matrix <- df_matrix %>%
    tibble::rownames_to_column(var = "ensembl_gene_id")
    # filter to remove genes without a gene symbol
    gene_info <- gene_info %>% dplyr::filter(complete.cases(.))
    # add gene symbols to expression df
    df_matrix <- dplyr::inner_join(gene_info, df_matrix, 
                                by = "ensembl_gene_id")
    df_matrix <- df_matrix %>%
        dplyr::filter(hgnc_symbol != '') %>%
        distinct(hgnc_symbol, .keep_all = TRUE) %>%
        tibble::column_to_rownames(var = "hgnc_symbol") %>%
        subset(select = -c(ensembl_gene_id))

    matrix <- as.matrix(df_matrix)

    end_col <- start_col + ncol(matrix) - 1
    h5write(matrix, recount3_h5, "count", 
            index = list(1:total_rows, start_col:end_col))
    start_col <- end_col + 1
    all_colnames <- c(all_colnames, colnames(matrix))
}

file '/home/msubirana/Documents/pivlab/plier_recount3/output/nbs/create_recount3_model/recount3_dataset_tpm.h5' already exists.

2024-06-21 18:36:15.440184 downloading and reading the metadata.



2024-06-21 18:36:15.793949 caching file sra.sra.SRP107565.MD.gz.

2024-06-21 18:36:16.341716 caching file sra.recount_project.SRP107565.MD.gz.

2024-06-21 18:36:16.906145 caching file sra.recount_qc.SRP107565.MD.gz.

2024-06-21 18:36:17.435856 caching file sra.recount_seq_qc.SRP107565.MD.gz.

Warning message:
“The 'url' <http://duffel.rail.bio/recount3/human/data_sources/sra/metadata/65/SRP107565/sra.recount_pred.SRP107565.MD.gz> does not exist or is not available.”
2024-06-21 18:36:17.927135 downloading and reading the feature information.

2024-06-21 18:36:18.315335 caching file human.gene_sums.G026.gtf.gz.

2024-06-21 18:36:18.761522 downloading and reading the counts: 216 samples across 63856 features.

2024-06-21 18:36:19.109067 caching file sra.gene_sums.SRP107565.G026.gz.

2024-06-21 18:36:19.689796 constructing the RangedSummarizedExperiment (rse) object.

2024-06-21 18:36:22.699106 downloading and reading the metadata.

2024-06-21 18:36:23.109608 caching file sra.sra.SRP107565

In [20]:
all_rownames  <- rownames(initial_matrix)
dimnames_rds <- file.path(output_nb_path, "dimnames.RDS")
saveRDS(list(row.names = all_rownames, col.names = all_colnames), file = dimnames_rds)

In [21]:
recount3_h5 <- file.path(output_nb_path, "recount3_dataset_tpm.h5")
dimnames_rds <- file.path(output_nb_path, "dimnames.RDS")

sce <- DelayedArray(seed = HDF5ArraySeed(filepath = recount3_h5, name = "count"))
dimnamaes <- readRDS(dimnames_rds)
rownames(sce) <- dimnamaes$row.names
colnames(sce) <- dimnamaes$col.names

sce[is.na(sce)] <- 0
sce <- sce[which(DelayedMatrixStats::rowSds(sce) >0),]

sce

<38447 x 661> DelayedMatrix object of type "double":
         SRR5579425  SRR5579426  SRR5579433 ... SRR7280714 SRR7280715
  SCYL3  7.60426358  8.21645078  8.64601075   .  6.3227423  5.2750730
  FIRRM 12.19380568 11.15075743  8.95064922   . 14.5106309 10.7818084
    FGR  0.00000000  0.00000000  0.00000000   .  0.0000000  0.0785297
    CFH  0.07174062  0.00000000  0.00000000   .  8.7081807  7.1243656
  STPG1  1.96629475  1.70422297  1.39559122   .  3.2590608  3.2634849
    ...           .           .           .   .          .          .
MIR24-1    4.124663    5.854525    0.000000   .   1.132107   0.000000
 MIR363    0.000000    0.000000    0.000000   .   0.000000   0.000000
MIR92A2    0.000000    0.000000    0.000000   .   0.000000   0.000000
MIR2861    0.000000    0.000000    0.000000   .   0.000000   0.000000
 MIR223    0.000000    0.000000    0.000000   .   0.000000   0.000000

# Prepare data for all the models

In [22]:
all_paths <- PLIER::combinePaths(bloodCellMarkersIRISDMAP, svmMarkers, canonicalPathways, chemgenPathways, oncogenicPathways,
                                 xCell, immunePathways, chr21_pathway)

Keep all the genes present in RNA-seq matrix adding 0 if not present in pathways and remove the once not present in RNA-seq matrix

In [23]:
# Extract rownames from sce_symbol_filter
gene_names <- rownames(sce)

# Convert all_paths to a data frame if it's not already
all_paths_df <- as.data.frame(all_paths)

# Filter all_paths to retain only rows present in sce_symbol_filter
filtered_all_paths <- all_paths_df[rownames(all_paths_df) %in% gene_names, ]

# Identify genes in sce_symbol_filter not present in all_paths
missing_genes <- setdiff(gene_names, rownames(filtered_all_paths))

# Create a data frame of missing genes with all zeros
missing_genes_df <- as.data.frame(matrix(0, nrow=length(missing_genes), ncol=ncol(all_paths_df)))
rownames(missing_genes_df) <- missing_genes
colnames(missing_genes_df) <- colnames(all_paths_df)

# Combine filtered_all_paths and missing_genes_df
combined_paths <- rbind(filtered_all_paths, missing_genes_df)

# Reorder rows to match the order of genes in sce_symbol_filter
final_all_paths <- combined_paths[gene_names, ]

final_all_paths <- combined_paths[match(gene_names, rownames(combined_paths)), ]

final_all_paths <- as.matrix(final_all_paths)

# Print the resulting data frame
head(final_all_paths)

,IRIS_Bcell-Memory_IgG_IgA,IRIS_Bcell-Memory_IgM,IRIS_Bcell-naive,IRIS_CD4Tcell-N0,IRIS_CD4Tcell-Th1-restimulated12hour,IRIS_CD4Tcell-Th1-restimulated48hour,IRIS_CD4Tcell-Th2-restimulated12hour,IRIS_CD4Tcell-Th2-restimulated48hour,IRIS_CD8Tcell-N0,IRIS_DendriticCell-Control,⋯,chr21_q21.1,chr21_q21.2,chr21_q21.3,chr21_q22.11,chr21_q22.12,chr21_q22.2,chr21_q22.13,chr21_q22.3,chr21_p12,chr21
SCYL3,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
FIRRM,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
FGR,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
CFH,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
STPG1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
NIPAL3,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


# Run delayedPLIER

In [24]:
setAutoRealizationBackend("HDF5Array") #supportedRealizationBackends(), getRealizationBackend()

# Assign arguments to variables 
output_file_recount3 <- file.path(output_nb_path, 'recount3_model.rds')
output_dplier <- file.path(output_nb_path, 'recount3_model_r')

dir.create(output_dplier, showWarnings = F)

# Run PLIER (with common genes)
delayedPlier_result=.GlobalEnv$PLIER(data=sce, priorMat=final_all_paths, output_path = output_dplier)

# Prepare output directory
output_file_path=dirname(output_file_recount3)
dir.create(dirname(output_file_path), showWarnings = FALSE, recursive = TRUE)

# Save results
saveRDS(delayedPlier_result, file = output_file_recount3)

Removing 471 pathways with too few genes

Computing SVD

Using rsvd

Done

k is set to 160



[1] 121.9623
[1] "L2 is set to 121.96230977279"
[1] "L1 is set to 60.9811548863949"


errorY (SVD based:best possible) = 0.5415

New L3 is 9.61116520613947e-05

New L3 is 8.48182352464692e-05

New L3 is 0.000108908769855066

New L3 is 8.48182352464692e-05

Bdiff is not decreasing

New L3 is 9.61116520613947e-05

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

Bdiff is not decreasing

converged at  iteration 104 Bdiff is not decreasing

There are 86  LVs with AUC>0.70



ERROR: Error in H5Fcreate(file): HDF5. File accessibility. Unable to open file.


# Save as pickle

In [ ]:
library(reticulate)

save_as_pickle <- function(object, filename, save_directory) {
  full_path <- file.path(save_directory, filename)
  py_save_object(r_to_py(object), full_path)
}

PLIER_model_to_pickle = function(PLIER_model, save_directory){
    
    # Check if the directory exists, create if it does not
    if (!dir.exists(save_directory)) {
      dir.create(save_directory, recursive = TRUE)
    }
    
    # Assuming gtex_tmp_1 is a list with various data types
    names_list <- names(PLIER_model)
    
    for (name in names_list) {
      element <- PLIER_model[[name]]
      if (is.matrix(element) || is.array(element)) {
        # Convert matrices/arrays to data frames before saving
        df <- as.data.frame(element)
        save_as_pickle(df, paste0(name, ".pkl"), save_directory)
      } else {
        # Save other data types directly
        save_as_pickle(element, paste0(name, ".pkl"), save_directory)
      }
    }  
}